In [7]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import numpy as np
import pandas as pd
from time import sleep
from datetime import datetime
import locale

In [ ]:
options = webdriver.ChromeOptions()
options.add_argument("--headless=new")
driver = webdriver.Chrome(options = options)

driver.get("https://pantip.com/forum/lumpini")


target_element = driver.find_elements(By.CLASS_NAME, 'col-12')
target_element


found_element = None
for i in target_element:
    #print(i.text)
    #print('___')
    if "Pantip Trend" in i.text:
       # print("found")
        found_element = i
        title_e = found_element.find_elements(By.CLASS_NAME, 'pt-list-item__title')
        name_e = found_element.find_elements(By.TAG_NAME, 'h5')
        date_e = found_element.find_elements(By.XPATH, ".//span[@title]")

        url_elements = found_element.find_elements(By.XPATH, ".//a[contains(@href, 'https://pantip.com/topic')]")

        dt_lst = []
        
                # Set locale to Thai for date parsing
        try:
            locale.setlocale(locale.LC_ALL, 'th_TH.UTF-8')
        except locale.Error:
            print("Warning: Could not set locale to 'th_TH.UTF-8'. Date parsing might fail for Thai month names.")
            # Fallback for some systems
            try:
                 locale.setlocale(locale.LC_ALL, 'Thai_Thailand.65001')
            except locale.Error:
                 print("Warning: Could not set locale to 'Thai_Thailand.65001'. Date parsing might fail.")


        for dt in date_e:
            try:
                dt = dt.get_attribute('title')
            # Remove " เวลา " and " น" for easier parsing
                dt_string_cleaned = dt.replace(" เวลา ", " ").replace(" น.", "")

                # Parse the date string
                # %d: day, %B: full month name, %Y: year, %H: hour (24-hour), %M: minute
                # Note: %Y here expects a Gregorian year. We'll adjust for Buddhist year.
                parsed_dt = datetime.strptime(dt_string_cleaned, '%d %B %Y %H:%M')

                # Adjust for Buddhist calendar year (2568 is ~2025 Gregorian)
                # The difference is typically 543 years.
                # We parse it as Gregorian and then subtract 543 from the parsed year.
                # This assumes the parsed year (e.g., 2568) was intended as Buddhist.
                gregorian_year = parsed_dt.year - 543
                # Create a new datetime object with the corrected Gregorian year
                dt_object = parsed_dt.replace(year=gregorian_year)

                dt_lst.append(dt_object)
            except ValueError as e:
                print(f"Error parsing date string '{dt}': {e}")
                dt_lst.append(None) # Append None if parsing fails
            # --- End Date Parsing ---
            #dt_lst.append(dt.get_attribute('title'))
        #full_datetime = date_e.get_attribute("title")
        title_lst = []
        name_lst = []
        #date_lst = []
        for title, name in zip(title_e, name_e):
            (title_lst.append(title.text))
            (name_lst.append(name.text))
            #(date_lst.append(date.text))
            
        urls =[]
        for url_element in url_elements:
            
            urls.append(url_element.get_attribute("href"))

        urls = list(set(urls))


df = pd.DataFrame({
"title": title_lst, "name": name_lst,
"date": dt_lst, "url": urls
})



[<selenium.webdriver.remote.webelement.WebElement (session="734c63ae0fe6b87c3769828996cd2960", element="f.2D555CB9CA6AA7ACD06DAAC1DCED4DA6.d.E352171C147FA12AE1738B04DBE853D7.e.15")>,
 <selenium.webdriver.remote.webelement.WebElement (session="734c63ae0fe6b87c3769828996cd2960", element="f.2D555CB9CA6AA7ACD06DAAC1DCED4DA6.d.E352171C147FA12AE1738B04DBE853D7.e.16")>,
 <selenium.webdriver.remote.webelement.WebElement (session="734c63ae0fe6b87c3769828996cd2960", element="f.2D555CB9CA6AA7ACD06DAAC1DCED4DA6.d.E352171C147FA12AE1738B04DBE853D7.e.17")>,
 <selenium.webdriver.remote.webelement.WebElement (session="734c63ae0fe6b87c3769828996cd2960", element="f.2D555CB9CA6AA7ACD06DAAC1DCED4DA6.d.E352171C147FA12AE1738B04DBE853D7.e.18")>]

[datetime.datetime(2025, 4, 15, 17, 9), datetime.datetime(2014, 12, 15, 21, 15), datetime.datetime(2025, 4, 18, 14, 53), datetime.datetime(2025, 4, 19, 5, 0), datetime.datetime(2025, 4, 12, 2, 8), datetime.datetime(2025, 4, 19, 10, 35), datetime.datetime(2025, 4, 16, 0, 26), datetime.datetime(2025, 4, 11, 12, 8), datetime.datetime(2025, 4, 9, 22, 40), datetime.datetime(2025, 4, 13, 7, 0)]


,title,name,date,url
0,สรุปแล้วการลดน้ำหนักควรกินอะไร หมู Vs ไก่,สมาชิกหมายเลข 2736330,2025-04-15 17:09:00,https://pantip.com/topic/32986685
1,เอาหม้อหุงข้าวมาตั้งไฟประกอบอาหาร มันไม่อันตรา...,wisdom of social,2014-12-15 21:15:00,https://pantip.com/topic/43387548
2,ทำไมรู้สึกว่ากินไก่ทอด+สลัด ลดน้ำหนักได้ดีการก...,สมาชิกหมายเลข 6809153,2025-04-18 14:53:00,https://pantip.com/topic/43379959
3,มีคำถามที่ค้างคาในใจแต่เล่าให้ใครฟังไม่ได้จริง...,สมาชิกหมายเลข 8755858,2025-04-19 05:00:00,https://pantip.com/topic/43401471
4,กุนเชียง มีสารก่อมะเร็v เทียบเท่าบุหรี่,สมาชิกหมายเลข 7749566,2025-04-12 02:08:00,https://pantip.com/topic/43393594
5,มนุษย์เป็นสัตว์กินพืชหรือกินเนื้อ?,สมาชิกหมายเลข 5740463,2025-04-19 10:35:00,https://pantip.com/topic/43383533
6,ทำไมคน เมกา ถึง อ้วนกันแทบทุกคนครับ ????,สมาชิกหมายเลข 4809088,2025-04-16 00:26:00,https://pantip.com/topic/43385459
7,เวลาเหนื่อยๆเพลียๆ เครื่องดื่มอะไรบ้างครับที่ช...,สมาชิกหมายเลข 941889,2025-04-11 12:08:00,https://pantip.com/topic/43401787
8,กูรูเฉลย อาหารร้อนๆ ต้องรอเย็นก่อนแช่ตู้เย็นไห...,stardom,2025-04-09 22:40:00,https://pantip.com/topic/43399971
9,"หมอไต้หวัน แนะนำ 9 อาหารจากซูเปอร์มาร์เก็ต ""ช่...",สมาชิกหมายเลข 2469579,2025-04-13 07:00:00,https://pantip.com/topic/43392494
